Loads and preprocesses images from the specified directory.

In [ ]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def load_data(directory):
    datagen = ImageDataGenerator(rescale=1./255)
    return datagen.flow_from_directory(directory, target_size=(224, 224), class_mode='categorical')

image_array = load_data('./input/animals10/raw-img')

One-hot encodes the labels for multi-class classification.

In [ ]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder

labels = np.array([0, 1, 2])  # Example label array
encoder = OneHotEncoder(sparse=False)
image_label = encoder.fit_transform(labels.reshape(-1, 1))

Splits the dataset into training and testing sets.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(image_array, image_label, test_size=0.2, random_state=42)

Defines a model using EfficientNetB7 as the base.

In [ ]:
from tensorflow.keras.applications import EfficientNetB7
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

model = Sequential([
    EfficientNetB7(input_shape=(224, 224, 3), include_top=False, weights='imagenet'),
    Flatten(),
    Dense(256, activation='relu'),
    Dense(len(np.unique(labels)), activation='softmax')
])

Compiles the model with the optimizer and loss function.

In [ ]:
from tensorflow.keras.optimizers import Adam
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

Trains the model with specified epochs, batch size, and callbacks.

In [ ]:
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=15, batch_size=32, callbacks=[EarlyStopping(monitor='val_loss'), ModelCheckpoint('best_model.h5', save_best_only=True), ReduceLROnPlateau(monitor='val_loss')])

Evaluates the model performance on the test set.

In [ ]:
evaluation = model.evaluate(X_test, y_test)